In [20]:
import os
import pandas as pd

train_data_path ="data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [21]:
train_data.groupby('class').count()

,idx,conversation
class,,
갈취 대화,981,981
기타 괴롭힘 대화,1094,1094
직장 내 괴롭힘 대화,979,979
협박 대화,896,896


In [22]:
train_data[train_data.iloc[392]['conversation'] == train_data['conversation']]

,idx,class,conversation
29,29,기타 괴롭힘 대화,병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 ...
392,392,기타 괴롭힘 대화,병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 ...


In [23]:
train_data.duplicated('conversation').sum()

104

In [24]:
train_data = train_data.drop_duplicates(subset='conversation')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3846 entries, 0 to 3949
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3846 non-null   int64 
 1   class         3846 non-null   object
 2   conversation  3846 non-null   object
dtypes: int64(1), object(2)
memory usage: 120.2+ KB


In [25]:
train_data.groupby('class').count()

,idx,conversation
class,,
갈취 대화,973,973
기타 괴롭힘 대화,1011,1011
직장 내 괴롭힘 대화,970,970
협박 대화,892,892


In [26]:
train_data.iloc[29,2]

'병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 되지 너 공부 개못하는 찐따잖아.\n야 닭대가리가 말도 하네 동물농장 나가자.\n너네 학교폭력으로 신고하기 전에 그만 해.\n아이고 무서워라 살려주세요 으헝헝\n제발 좀 저리 가.\n나였으면 아이스크림 먹을 시간에 공부한다.\n멈춰!\n저러니까 쳐맞고 다니지'

In [27]:
import re

def preprocess_document(document):
    document = re.sub('[^ㄱ-ㅎ가-힣a-zA-Z0-9]', ' ', document)
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


preprocess_document(train_data.iloc[1,2])

'길동경찰서입니다 9시 40분 마트에 폭발물을 설치할거다 네 똑바로 들어 한번만 더 얘기한다 장난전화 걸지 마시죠 9시 40분 마트에 폭발물이 터지면 다 죽는거야 장난전화는 업무방해죄에 해당됩니다 판단은 너에게 달려있다 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지 선생님 진정하세요 난 이야기했어 경고했다는 말이야'

In [9]:
from konlpy.tag import Mecab

mecab = Mecab()

document = preprocess_document(train_data.iloc[1,2])
mecab.morphs(document)

['길동',
 '경찰서',
 '입니다',
 '9',
 '시',
 '40',
 '분',
 '마트',
 '에',
 '폭발물',
 '을',
 '설치',
 '할',
 '거',
 '다',
 '네',
 '똑바로',
 '들',
 '어',
 '한',
 '번',
 '만',
 '더',
 '얘기',
 '한다',
 '장난',
 '전화',
 '걸',
 '지',
 '마시',
 '죠',
 '9',
 '시',
 '40',
 '분',
 '마트',
 '에',
 '폭발물',
 '이',
 '터지',
 '면',
 '다',
 '죽',
 '는',
 '거',
 '야',
 '장난',
 '전화',
 '는',
 '업무',
 '방해죄',
 '에',
 '해당',
 '됩니다',
 '판단',
 '은',
 '너',
 '에게',
 '달려',
 '있',
 '다',
 '길동',
 '경찰서',
 '에',
 '도',
 '폭발물',
 '터지',
 '면',
 '꽤',
 '나',
 '재미있',
 '겠',
 '지',
 '선생',
 '님',
 '진정',
 '하',
 '세요',
 '난',
 '이야기',
 '했',
 '어',
 '경고',
 '했',
 '다는',
 '말',
 '이',
 '야']

In [10]:
class_names = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

train_data = train_data[['class', 'conversation']]
train_data['class'] = train_data['class'].map(lambda c: class_names.index(c))

train_data.head()

,class,conversation
0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(preprocessor=preprocess_document, tokenizer=mecab.morphs)
print(vector.fit_transform(train_data['conversation']).toarray())
print(vector.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'지금': 10613, '너': 2274, '스스로': 7028, '를': 4047, '죽여': 10453, '달': 2800, '라고': 3869, '애원': 7874, '하': 12255, '는': 2564, '것': 628, '인가': 9450, '아닙니다': 7584, '죄송': 10374, '합니다': 12453, '죽': 10452, '을': 9184, '거': 467, '면': 4447, '혼자': 12782, '지': 10602, '우리': 8830, '까지': 1660, '사건': 6129, '에': 8247, '휘말리': 12935, '게': 634, '해': 12477, '진짜': 10803, '버리': 5251, '고': 765, '싶': 7365, '정말': 10135, '잘못': 9793, '했': 12574, '습니다': 7070, '가': 106, '선택': 6552, '을래': 9193, '네': 2318, '가족': 217, '줄까': 10500, '에게': 8248, '권': 1190, '이': 9248, '없': 8222, '어': 7969, '못': 4602, '한다면': 12367, '와': 8674, '모조리': 4544, '버릴': 5256, '야': 7893, '겠': 649, '한': 12336, '번': 5285, '만': 4175, '도와': 3128, '주': 10384, '세요': 6653, '그냥': 1249, '다': 2635, '버려야': 5243, '군': 1144, '의': 9224, '제발': 10213, '길동': 1563, '경찰서': 724, '입니다': 9613, '9': 103, '시': 7088, '40': 6

In [12]:
print(mecab.morphs(document))

print('[', end='')
for i in vector.transform([document]).toarray()[0]:
    print(i, end='. ')
print('\b]')

['길동', '경찰서', '입니다', '9', '시', '40', '분', '마트', '에', '폭발물', '을', '설치', '할', '거', '다', '네', '똑바로', '들', '어', '한', '번', '만', '더', '얘기', '한다', '장난', '전화', '걸', '지', '마시', '죠', '9', '시', '40', '분', '마트', '에', '폭발물', '이', '터지', '면', '다', '죽', '는', '거', '야', '장난', '전화', '는', '업무', '방해죄', '에', '해당', '됩니다', '판단', '은', '너', '에게', '달려', '있', '다', '길동', '경찰서', '에', '도', '폭발물', '터지', '면', '꽤', '나', '재미있', '겠', '지', '선생', '님', '진정', '하', '세요', '난', '이야기', '했', '어', '경고', '했', '다는', '말', '이', '야']


[0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.26824035986140476. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.253083161169039. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0.

. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0

. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.15672336544197168. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.

0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.07481637997671724. 0.0. 0.0. 0.0. 0.050547970789830185. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.

. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.1531174635005071. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.12366033083456357. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.

. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0. 0.0

In [13]:
from sklearn.model_selection import train_test_split

x_train = vector.transform(train_data['conversation'])
y_train = train_data['class']

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [14]:
print('train size:', x_train.shape[0])
print('validation size:', x_val.shape[0])

train size: 3076
validation size: 770


In [15]:
from sklearn.naive_bayes import ComplementNB

model = ComplementNB()
model.fit(x_train, y_train)

ComplementNB()

In [16]:
from sklearn.metrics import classification_report

print(classification_report(y_train, model.predict(x_train), zero_division=0))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       697
           1       0.92      0.96      0.94       760
           2       0.92      1.00      0.96       819
           3       0.98      0.89      0.93       800

    accuracy                           0.94      3076
   macro avg       0.95      0.94      0.94      3076
weighted avg       0.95      0.94      0.94      3076



In [17]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def graph_confusion_matrix(model, x_test, y_test, classes_name):
    df_cm = pd.DataFrame(confusion_matrix(y_test, model.predict(x_test)), index=classes_name, columns=classes_name)
    fig = plt.figure(figsize=(12,12))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=12)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=12)
    plt.ylabel('label')
    plt.xlabel('predicted value')
    plt.show()
    
graph_confusion_matrix(model, x_train, y_train, class_names)

NameError: name 'plt' is not defined

In [ ]:
print(classification_report(y_val, model.predict(x_val), zero_division=0))

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

In [ ]:
graph_confusion_matrix(model, x_val, y_val, class_names)

In [ ]:
test_data = pd.read_json('data/test.json', orient='index')

test_data.head()

In [ ]:
test_data.info()

In [ ]:
submission_data = []

for file_name, row in test_data.iterrows():
    text = row['text']
    _input = vector.transform([text])
    predicted = model.predict(_input)[0]
    
    submission_data.append({
        'file_name': file_name,
        'class': predicted
    })

In [ ]:
submission_data = pd.DataFrame(submission_data)

submission_data.head()

In [ ]:
submission_data.to_csv('submission.csv', index=False)